In [ ]:
!nvcc --version
!pip3 install pycuda

In [3]:
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import pycuda.autoinit
import numpy as np

In [52]:
kernel_code = """
#include <curand_kernel.h>

extern "C" __global__ void generate_random_numbers(float* numbers, int n, int seed) {
  int idx = threadIdx.x + blockIdx.x * blockDim.x;

  if (idx < n) {
    curandState state;
    curand_init(seed, idx, 0, &state);
    numbers[idx] = curand_uniform(&state);
  }
}

extern "C" __global__ void debug_func(void) {
  printf("Debug print");
}

extern "C" __global__ void calc_positional_encoding(float* pos_enc, int num_rows, int num_cols) {
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;

  if (row < num_rows && col < num_cols) {
    int idx = row * num_cols + col;
    pos_enc[idx] = 1;
  } else {
    printf("Out of bounds!!!");
  }

  // pos_enc = (current_dim & 1) ? sin(token_idx_in_sentence / (10000 * pow((2 * current_dim) / embedding_dimension) :
  //                              cos(token_idx_in_sentence / (10000 * pow((2 * current_dim) / embedding_dimension)));
}
"""

mod = SourceModule(kernel_code,
                   no_extern_c=True,  # This is important!
                   options=["-std=c++11",
                           "-Xcompiler",
                           "-fPIC"])

In [27]:
embedding_dimension = 2 # num of dimensions in each vector in the embedding matrix

In [54]:
# Create Positional Encoding
# embedding_dimension
pos_encodings_batch_size = 512
pos_encodings_num_elements = pos_encodings_batch_size * embedding_dimension
pos_encodings_size_bytes = pos_encodings_num_elements * np.float32().nbytes
pos_encodings_gpu = cuda.mem_alloc(pos_encodings_size_bytes)

calc_positional_encoding = mod.get_function("calc_positional_encoding")
calc_positional_encoding(pos_encodings_gpu, np.int32(pos_encodings_batch_size), np.int32(embedding_dimension), block=(embedding_dimension, pos_encodings_batch_size, 1))

pos_encodings_host = np.empty(pos_encodings_num_elements, dtype=np.float32)
cuda.memcpy_dtoh(pos_encodings_host, pos_encodings_gpu)

# np.set_printoptions(threshold=np.inf)
print(f"{pos_encodings_host=}")
print(pos_encodings_host.shape)

pos_encodings_host=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
      

In [29]:
# Embedding matrix : (token, vector dimensions)

sentence = "This is a sentence"
vocab = ["This", "is", "a", "sentence"]
sentence_toks = [0, 1, 2, 3] # Straight forward
word2tok = {"This" : 0, "is" : 1, "a" : 2, "sentence" : 3}

# Create the embedding matrix
vocab_size = len(vocab)
embedding_num_elements = vocab_size * embedding_dimension
embedding_size_bytes = embedding_num_elements * np.float32().nbytes
embedding_matrix_gpu = cuda.mem_alloc(embedding_size_bytes)

generate_random_numbers = mod.get_function("generate_random_numbers")
generate_random_numbers(embedding_matrix_gpu, np.int32(embedding_num_elements), np.int32(0), block=(256, 1, 1), grid=(int(np.ceil(embedding_num_elements / 256)), 1))

embedding_matrix_host = np.empty(embedding_num_elements, dtype=np.float32)
cuda.memcpy_dtoh(embedding_matrix_host, embedding_matrix_gpu)

print(embedding_matrix_host)

# We need to build a method to lookup tokens in the embedding matrix
# Use word2tok

# debug_func = mod.get_function("debug_func")
# debug_func(block=(2, 2, 2))


[0.74021935 0.9209938  0.03902049 0.9689629  0.92514056 0.4463501
 0.6673192  0.10993068]
